In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

# Specify the path of the password-protected zip file you want to extract
zip_file_path = "/content/drive/MyDrive/data/VeRi-EEEM071 (2).zip"

# Specify the password for the zip file
password = "sUrreY_eeEm071_VeRi"

# Create a ZipFile object and extract its contents to a specified directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/new_data", pwd=password.encode('utf-8'))

The Google Drive API credentials are set up and a connection to Google Drive is established. The path to a folder containing XML files is specified, as well as the path to the folder where the resulting CSV files will be saved. The code then iterates over each XML file in the specified folder. For each XML file, it parses the file and creates a corresponding CSV file. The XML headers are extracted and written as the headers in the CSV file. The XML data is extracted and written as rows in the CSV file. Finally, the generated CSV file is uploaded to Google Drive and the ID of the uploaded file is printed.

In [ ]:
import os
import csv
import xml.etree.ElementTree as ET
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.client import GoogleCredentials

# Set up the Google Drive API credentials
auth.authenticate_user()
creds = GoogleCredentials.get_application_default()
drive_service = build('drive', 'v3', credentials=creds)

# # Mount Google Drive
# drive.mount('/content/drive')

# Set the path to the folder containing the XML files in Google Drive
xml_folder_path = '/content/drive/MyDrive/new_data/train_label.xml'

# Set the path to the folder where the CSV files will be saved in Google Drive
csv_folder_path = '/content/drive/MyDrive/new_data/VeRi/train_label.csv'

# Iterate over all XML files in the specified folder
for filename in os.listdir(xml_folder_path):
  if filename.endswith('.xml'):
    # Parse the XML file
    xml_tree = ET.parse(xml_folder_path + filename)
    root = xml_tree.getroot()

    # Create a CSV file with the same name as the XML file
    csv_filename = os.path.splitext(filename)[0] + '.csv'
    csv_file = open(csv_folder_path + csv_filename, 'w', newline='')
    csv_writer = csv.writer(csv_file)

    # Write the headers to the CSV file
    headers = []
    for child in root[0]:
      headers.append(child.tag)
    csv_writer.writerow(headers)

    # Write the data to the CSV file
    for child in root:
      row = []
      for element in child:
        row.append(element.text)
      csv_writer.writerow(row)

    # Close the CSV file
    csv_file.close()

    # Upload the CSV file to Google Drive
    file_metadata = {'name': csv_filename, 'parents': [drive_service.files().get(fileId='root').execute()['id']]}
    media = {'mimeType': 'text/csv', 'body': open(csv_folder_path + csv_filename, 'rb')}
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('CSV file ID: %s' % file.get('id'))

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

Defining a transformation pipeline for images. It resizes the images to 224x224 pixels, converts them to tensors, and applies normalization to bring the pixel values within the range of -1 to 1.

In [ ]:
# Define the transforms to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
import xml.etree.ElementTree as ET

with open('/content/drive/MyDrive/new_data/VeRi/train_label.xml','r') as f:
   ef = ET.fromstring(f.read())

Defining a custom dataset class called CustomDataset that extends the Dataset class from PyTorch. It takes a root directory, a labels file, and an optional transformation as input. The dataset's length is determined by the number of labels. The __getitem__ method retrieves an image and its corresponding label by parsing the labels file. The image is loaded, converted to RGB, and transformed if the transform is specified. Finally, the method returns the transformed image and its label. The _parse_labels method reads the labels file, parses the XML content, and returns a list of dictionaries containing image names and labels.

In [ ]:
import os
import xml.etree.ElementTree as ET
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir, labels_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = self._parse_labels(labels_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.labels[idx]['img_name'])
        img = Image.open(img_path).convert('RGB')
        label = self.labels[idx]['label']
        if self.transform:
            img = self.transform(img)
        return img, label

    def _parse_labels(self, labels_file):
        with open(labels_file, 'r', encoding='utf-8') as f:
          tree = ET.parse(f.read())
          root = tree.getroot()
          labels = []
        for child in root:
            if child.tag != 'image':
                continue
            img_name = child.attrib['name']
            label = int(child.find('label').text)
            labels.append({'img_name': img_name, 'label': label})
        return labels

A dataset and dataloader are created using the CustomDataset class. The root_dir specifies the directory where the images are located, and the labels_file contains the corresponding labels for each image. The dataset is transformed using the defined transform object, and the dataloader is set to batch size 256, shuffling the data and using 4 workers for parallel data loading.

### Model and Training Setup:


*   A ResNet-18 model is created and customized for classification by replacing the last fully connected layer.
*   The model is configured with an output size of 10, assuming 10 classes.


*   The loss function is defined as cross-entropy.
*   The optimizer used is stochastic gradient descent (SGD) with a learning rate of 0.001 and momentum of 0.9.

### Training Process:

 

*   The model is trained for 30 epochs.
*   For each epoch, the running loss is initialized to 0.0.
*   The training data is iterated through in batches.
*   Inputs and labels are retrieved for each batch.
*   Optimizer gradients are set to zero.
*   List item
*   The forward pass is performed to obtain the outputs.
*   The loss is calculated using the cross-entropy loss function.
*   Backpropagation is applied, and the optimizer updates the model's parameters.
*   The running loss is accumulated.
*   Every 10 mini-batches, the average loss is printed.
*   After training, the message "Finished Training" is displayed.






In [ ]:
# create dataset and dataloader
root_dir = '/content/drive/MyDrive/new_data/VeRi/image_train'
labels_file = '/content/drive/MyDrive/new_data/VeRi/train_label.xml'
dataset = CustomDataset(root_dir=root_dir, labels_file=labels_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)

# create model and define loss function and optimizer
model = models.resnet18()
model.fc = nn.Linear(512, 10)  # assuming 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train model
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get inputs and labels
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

# evaluate model on test data
# define test dataset and dataloader in the same way as train data

# set model to evaluation mode
# model.eval()

## Testing Resnet 18

In [ ]:
# define transformations to be applied to each image
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# define custom test dataset class
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, labels_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = self._parse_labels(labels_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels[idx]['img_name']
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def _parse_labels(self, labels_file):
        tree = ET.parse(labels_file)
        root = tree.getroot()
        labels = []
        for child in root:
            if child.tag != 'image':
                continue
            img_name = child.attrib['name']
            label = int(child.find('label').text)
            labels.append({'img_name': img_name, 'label': label})
        return labels

# create test dataset and dataloader
test_root_dir = '/content/drive/MyDrive/new_data/VeRi/image_test'
test_labels_file = '/content/drive/MyDrive/new_data/VeRi/test_label.xml'
test_dataset = CustomTestDataset(test_root_dir, test_labels_file, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

# set model to evaluation mode
model.eval()

# define variables to keep track of total and correct predictions
total = 0
correct = 0

# iterate over test data
with torch.no_grad():
    for data in test_dataloader:
        # get inputs and labels
        inputs, labels = data

        # forward pass
        outputs = model(inputs)

        # get predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # update total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# calculate accuracy on test data
accuracy = 100 * correct / total
print('Accuracy on test data: {:.2f}%'.format(accuracy))

## fitting vgg 16

In [ ]:
# define transformations to be applied to each image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, labels_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = self._parse_labels(labels_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels[idx]['img_name']
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def _parse_labels(self, labels_file):
        tree = ET.parse(labels_file)
        root = tree.getroot()
        labels = []
        for child in root:
            if child.tag != 'image':
                continue
            img_name = child.attrib['name']
            label = int(child.find('label').text)
            labels.append({'img_name': img_name, 'label': label})
        return labels

# create dataset and dataloader
root_dir = '/content/drive/MyDrive/new_data/VeRi/train_label.xml'
labels_file = '/content/drive/MyDrive/new_data/VeRi/test_label.xml'
dataset = CustomDataset(root_dir, labels_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)

# create model and define loss function and optimizer
model = models.vgg16_bn()
model.classifier[6] = nn.Linear(4096, 10)  # assuming 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train model
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get inputs and labels
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

## testing vgg 16

In [ ]:
# define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, labels_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = self._parse_labels(labels_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels[idx]['img_name']
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def _parse_labels(self, labels_file):
        tree = ET.parse(labels_file)
        root = tree.getroot()
        labels = []
        for child in root:
            if child.tag != 'image':
                continue
            img_name = child.attrib['name']
            label = int(child.find('label').text)
            labels.append({'img_name': img_name, 'label': label})
        return labels

# create dataset and dataloader
root_dir = '/content/drive/MyDrive/new_data/VeRi/image_test'
labels_file = '/content/drive/MyDrive/new_data/VeRi/test_label.xml'
dataset = CustomDataset(root_dir, labels_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)

# create model and define loss function and optimizer
model = models.vgg16()
model.classifier[-1] = nn.Linear(4096, 10)  # assuming 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train model
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get inputs and labels
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

# evaluate model on test data
# define test dataset and dataloader in the same way as train data

# set model to evaluation mode
model.eval()

# define variables to keep track of total and correct predictions
total = 0
correct = 0

# iterate over test data
for data in test_dataloader:
    # get inputs and labels
    inputs, labels = data

    # forward pass
    outputs = model(inputs)


    # update total and correct predictions
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

# calculate accuracy on test data
accuracy = 100 * correct / total
print('Accuracy on test data: {:.2f}%'.format(accuracy))




